# TFM Modelo de cine: Recomendador  | Grupo 1 | Carlos Onrubia y Guillem Balagué

#1. Librerías

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import polars as pl
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
import matplotlib.pyplot as plt
from typing import List, Sequence, Tuple

!pip install -U faiss-cpu
import faiss

import kagglehub
import dask.dataframe as dd
!pip install dask-sql --no-build-isolation
import dask.dataframe as dd
from dask_sql import Context
from dask.distributed import Client
import time
import gc
import pyarrow as pa
import pyarrow.parquet as pq
import os
import shutil
import ast
import seaborn as sns
import pandas as pd
import torch

import warnings
warnings.simplefilter(action = 'ignore')


import sklearn
from sklearn import set_config

from sklearn.tree import DecisionTreeClassifier

# transformers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder

# pipelines
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.metrics import accuracy_score
from sklearn.feature_extraction import FeatureHasher
import scipy.sparse as sp

from sentence_transformers import SentenceTransformer, models, util
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.4/18.4 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00


#2. Importación dataset de P2

In [ ]:
path_save='/content/drive/MyDrive/TFM_FILES'
file_name='df_limpio.csv'

Esta carepta será provisional para guardar diversos ficheros auxiliares:

In [ ]:
save_path = os.path.join(path_save, "recommender_artifacts")

In [ ]:
file_path = os.path.join(path_save, file_name)
if os.path.exists(file_path):
  df = pd.read_csv(file_path)
else:
  print(f"Archivo no encontrado: {file_path}")

In [ ]:
df.columns.tolist()

['tmdb_id',
 'tmdb_vote_average',
 'tmdb_vote_count',
 'tmdb_revenue',
 'tmdb_budget',
 'tmdb_original_language',
 'tmdb_original_title',
 'tmdb_popularity',
 'tmdb_production_companies',
 'tmdb_production_countries',
 'tmdb_spoken_languages',
 'imdb_startyear',
 'imdb_averagerating',
 'imdb_numvotes',
 'imdb_actor',
 'imdb_actress',
 'imdb_self',
 'imdb_writer',
 'imdb_director',
 'imdb_producer',
 'imdb_cinematographer',
 'imdb_editor',
 'imdb_production_designer',
 'imdb_composer',
 'runtime',
 'combined_genres']

In [ ]:
if 'tmdb_id' not in df.columns and 'id' in df.columns:
    df = df.rename(columns={'id': 'tmdb_id'})

df = df.set_index('tmdb_id')

In [ ]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 517409 entries, 2 to 1459582
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   tmdb_vote_average          517409 non-null  float64
 1   tmdb_vote_count            517409 non-null  int64  
 2   tmdb_revenue               517409 non-null  int64  
 3   tmdb_budget                517409 non-null  int64  
 4   tmdb_original_language     517409 non-null  object 
 5   tmdb_original_title        517409 non-null  object 
 6   tmdb_popularity            517409 non-null  float64
 7   tmdb_production_companies  517409 non-null  object 
 8   tmdb_production_countries  517409 non-null  object 
 9   tmdb_spoken_languages      517409 non-null  object 
 10  imdb_startyear             517409 non-null  int64  
 11  imdb_averagerating         517409 non-null  float64
 12  imdb_numvotes              517409 non-null  int64  
 13  imdb_actor                 517409

In [ ]:
df.head(2).T

tmdb_id,2,3
tmdb_vote_average,7.1,7.199
tmdb_vote_count,262,281
tmdb_revenue,0,0
tmdb_budget,0,0
tmdb_original_language,fi,fi
tmdb_original_title,Ariel,Varjoja paratiisissa
tmdb_popularity,8.155,5.946
tmdb_production_companies,villealfa filmproductions,villealfa filmproductions
tmdb_production_countries,finland,finland
tmdb_spoken_languages,finnish,english


In [ ]:
df_rec=df.copy()

#3 Funciones para calcular el recomendador

##3.1 Calculo join features

In [ ]:
feature_list=df_rec.columns.tolist()
feature_list

['tmdb_vote_average',
 'tmdb_vote_count',
 'tmdb_revenue',
 'tmdb_budget',
 'tmdb_original_language',
 'tmdb_original_title',
 'tmdb_popularity',
 'tmdb_production_companies',
 'tmdb_production_countries',
 'tmdb_spoken_languages',
 'imdb_startyear',
 'imdb_averagerating',
 'imdb_numvotes',
 'imdb_actor',
 'imdb_actress',
 'imdb_self',
 'imdb_writer',
 'imdb_director',
 'imdb_producer',
 'imdb_cinematographer',
 'imdb_editor',
 'imdb_production_designer',
 'imdb_composer',
 'runtime',
 'combined_genres']

In [ ]:
def build_joined_features(
    df: pd.DataFrame,
    cols: Sequence[str] = feature_list,
) -> pd.Series:
    """Devuelve una Serie con los metadatos concatenados y normalizados.

    Args:
        df: DataFrame con las columnas de entrada.
        cols: Lista de columnas que se desean concatenar. Por defecto se usa
              la constante global `feature_list`.
    Returns:
        pd.Series de strings lista para alimentar al modelo de texto.
    """
    # Convertir NaNs a cadena vacía y unir todo con un espacio
    text = (
        df[cols]
        .apply(lambda row: " ".join(str(valor) for valor in row if pd.notna(valor)), axis=1)
        .str.lower()
    )
    return text

##3.2 Embeddings

Función para pasar a valor numérico el texto encadenado.

In [ ]:
def compute_embeddings(
    texts: Sequence[str],
    model_name: str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    batch_size: int = 512,
    to_numpy: bool = True,
    normalize: bool = True,
) -> np.ndarray:
    """Codifica una lista de textos usando un modelo pre‑entrenado.

    Returns
    -------
    np.ndarray
        Array (num_samples, embedding_dim) normalizado.
    """
    model = SentenceTransformer(model_name)
    embeddings = model.encode(
        list(texts),
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=to_numpy,
        normalize_embeddings=normalize,
    )
    return embeddings.astype("float32")

##3.3 Similitud

Debido a tener más de 600.000 películas y después de investigar, encontramos que el índice FAISS es la forma más rápida (al estar basado en C++) para obtener los veccinos más próximos.

In [ ]:
def build_faiss_index(embeddings: np.ndarray) -> faiss.IndexFlatIP:
    """Crea y devuelve un índice FAISS (coseno = producto interno tras normalizar)."""
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    return index

##3.4 Obtener valores similares

Función para calcualr los más simialres. Para todas las películas tarda más de 30 minutos. Mejor guardarla y sólo "entrenarla" de vez en cuando.

In [ ]:
def precalc_similars(
    index: faiss.IndexFlatIP,
    embeddings: np.ndarray,
    top_k: int = 5,
) -> np.ndarray:
    """Devuelve una matriz (num_películas, top_k) con los índices de las películas más similares.

    Nota: solicitamos `top_k+1` porque la primera coincidencia suele ser la propia
    película, que se descarta.
    """
    distances, idxs = index.search(embeddings, top_k + 1)
    return idxs[:, 1:]

Añadir estas columnas al dataframe original.

In [ ]:
def add_similar_columns(df: pd.DataFrame, idxs: np.ndarray) -> pd.DataFrame:
    """Añade columnas `similar_1` … `similar_N` al DataFrame original y las devuelve. Son los íncides de cada película que ha encontrado"""
    out = df.copy()
    id_array = df.index.to_numpy()
    for j in range(idxs.shape[1]):
        out[f"similar_{j + 1}"] = id_array[idxs[:, j]]
    return out

##3.5 Funiones guardado y cargado

In [ ]:
def save_artifacts(
    path: str,
    df: pd.DataFrame,
    embeddings: np.ndarray,
    index: faiss.Index,
) -> None:
    """Guarda DataFrame, embeddings y el índice FAISS en `path`."""
    os.makedirs(path, exist_ok=True)

    df.to_parquet(os.path.join(path, "df_rec.parquet"))
    np.save(os.path.join(path, "embeddings.npy"), embeddings)
    faiss.write_index(index, os.path.join(path, "faiss.index"))

In [ ]:
def load_artifacts(path: str,
                   use_gpu: bool = False
    ) -> Tuple[pd.DataFrame, np.ndarray, faiss.Index]:

    """Carga los tres artefactos guardados en `save_artifacts`."""
    df = pd.read_parquet(os.path.join(path, "df_rec.parquet"))
    emb = np.load(os.path.join(path, "embeddings.npy"))
    index = faiss.read_index(os.path.join(path, "faiss.index"))
    if use_gpu and not isinstance(index, faiss.IndexGPU):
        res = faiss.StandardGpuResources()
        index = faiss.index_cpu_to_gpu(res, 0, index)
    return df, emb, index

#4. Función para ejecutar el modelo

Función más simple que en función del ya dataframe trabajado, te devuelve las películas más similares.

In [ ]:
def recommend(
    user_titles: Sequence[str],
    df: pd.DataFrame,
    index: faiss.IndexFlatIP,
    embeddings: np.ndarray,
    k: int = 5,
) -> pd.DataFrame:
    """Devuelve las *k* películas más afines al gusto de `user_titles`.

    Los títulos del usuario se buscan en la columna `tmdb_original_title`
    (minúsculas para evitar problemas de mayúsculas).  Excluye siempre las
    películas aportadas antes de devolver las recomendaciones.
    """
    if "tmdb_original_title" not in df.columns:
        raise KeyError(
            "El DataFrame no contiene la columna 'tmdb_original_title'. "
            "Asegúrate de que el nombre sea correcto."
        )

    # 1) Localizar las películas del usuario en el DataFrame
    titles_lower = df["tmdb_original_title"].astype(str).str.lower()
    mask = titles_lower.isin([t.lower() for t in user_titles])

    if not mask.any():
        raise ValueError(
            "Ninguno de los títulos proporcionados se encuentra en el catálogo."
        )

    user_idx = np.where(mask)[0]                       # posiciones 0‑based
    query_emb = embeddings[user_idx].mean(axis=0, keepdims=True)

    # 2) Buscar más vecinos de los necesarios para poder descartar los ya vistos
    _, idxs = index.search(query_emb, k + len(user_idx))

    # 3) Eliminar los que el usuario ya ha indicado y recortar a k resultados
    seen = set(user_idx)
    rec_indices = [idx for idx in idxs[0] if idx not in seen][:k]

    # 4) Devolver las k películas recomendadas como DataFrame limpio
    return df.iloc[rec_indices].reset_index(drop=True)

Función alternativa en base a texto libre

In [ ]:
def recommend_from_text(
    query_text: str,
    df: pd.DataFrame,
    index: faiss.Index,
    embeddings: np.ndarray,
    k: int = 5,
    model_name: str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
) -> pd.DataFrame:
    """Devuelve las *k* películas más parecidas a un texto libre (sin título).

    El texto se codifica con el **mismo modelo** usado para crear los embeddings
    del catálogo, garantizando la compatibilidad dimensional y semántica.
    """

    model = SentenceTransformer(model_name)
    model.max_seq_length = 128

    query_emb = model.encode([query_text], convert_to_numpy=True, normalize_embeddings=True)
    _, idxs = index.search(query_emb.astype("float32"), k)
    rec_indices = idxs[0].tolist()

    return df.iloc[rec_indices].reset_index(drop=True)

#5 Ejecución

##5.1 Funciones "grandes"

-> SÓLO ejecutar 1 vez a la semana.

In [ ]:
print("Construyendo columna joined_features…")
df_rec["joined_features"] = build_joined_features(df_rec)

print("Generando embeddings…")
emb = compute_embeddings(df_rec["joined_features"], batch_size=256)

print("Construyendo índice FAISS…")
ix = build_faiss_index(emb)

print("Pre‑calculando películas similares y guardando en DataFrame…")
sim_idxs = precalc_similars(ix, emb, top_k=5)
df_rec = add_similar_columns(df_rec, sim_idxs)

print(f"Guardando artefactos en {save_path}…")
save_artifacts(save_path, df_rec, emb, ix)

Construyendo columna joined_features…
Generando embeddings…


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2022 [00:00<?, ?it/s]

Construyendo índice FAISS…
Pre‑calculando películas similares y guardando en DataFrame…
Guardando artefactos en /content/drive/MyDrive/TFM_FILES/recommender_artifacts…


##5.2. Función pequeña

Función pequeña: Se puede ejecutar cuando se quiera!!

In [ ]:
print("Cargando artefactos…")
df_loaded, emb_loaded, ix_loaded = load_artifacts(save_path, use_gpu=False)

Cargando artefactos…


In [ ]:
test_films=[
    'Lost Highway',
    'Blue Velvet',
    'The Tree of Life',
    'Taxi Driver',
    'Casino'
]

In [ ]:
recomendaciones = recommend(
        test_films,
        df_loaded,
        ix_loaded,
        emb_loaded,
        k=5,
    )
recomendaciones[["tmdb_original_title", "imdb_startyear", "imdb_director", "imdb_writer","tmdb_production_countries","combined_genres","runtime"]]

,tmdb_original_title,imdb_startyear,imdb_director,imdb_writer,tmdb_production_countries,combined_genres,runtime
0,The Fanatic,2019,Fred Durst,Fred Durst,united states of america,crime,89
1,My Cousin Vinny,1992,Jonathan Lynn,Dale Launer,united states of america,crime,120
2,Fluke,1995,Carlo Carlei,James Herbert,united states of america,fantasy,96
3,Airheads,1994,Michael Lehmann,Rich Wilkes,united states of america,crime,92
4,Cool Hand Luke,1967,Stuart Rosenberg,Donn Pearce,united states of america,crime,127


In [ ]:
free_text="Estoy muy contento hoy en día y me gusta mucho Scorsese"

In [ ]:
recs_text = recommend_from_text(
        free_text,
        df_loaded,
        ix_loaded,
        emb_loaded,
        k=5
    )

recs_text[["tmdb_original_title", "imdb_startyear", "imdb_director", "imdb_writer","tmdb_production_countries","combined_genres","runtime"]]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

,tmdb_original_title,imdb_startyear,imdb_director,imdb_writer,tmdb_production_countries,combined_genres,runtime
0,Happy Moments,2018,Alexandros Zarbis,Alexandros Zarbis,greece,short,14
1,We Will Be Happy One Day,2011,Pawel Wysoczanski,Pawel Wysoczanski,poland,short,40
2,Happy End?!,2014,Petra Clever,Petra Clever,germany,romance,86
3,Happiness Ever After,2021,Thabang Moleya,Ayanda Halimana,south africa,romance,99
4,Smiles Leading To Happiness,2018,Jiro Enomoto,Jiro Enomoto,japan,not_informed,89
